In [1]:
!pip install transformers datasets accelerate jiwer torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.3 MB/s eta 0:00:00


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00


In [3]:
!pip install -U transformers datasets huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
!git clone https://github.com/facebookresearch/IMGUR5K-Handwriting-Dataset.git

Cloning into 'IMGUR5K-Handwriting-Dataset'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 48 (delta 16), reused 11 (delta 11), pack-reused 27 (from 1)
Receiving objects: 100% (48/48), 11.80 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, get_scheduler
from datasets import load_dataset, Dataset
import evaluate
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import os
import evaluate

from torch.optim import AdamW

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [7]:
from datasets import load_dataset


dataset = load_dataset("Teklia/IAM-line")


dataset = dataset["train"].train_test_split(test_size=0.1)
train_ds = dataset["train"]
test_ds = dataset["test"]


README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/73.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6482 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2915 [00:00<?, ? examples/s]

In [8]:
import json


with open("IMGUR5K-Handwriting-Dataset/dataset_info/imgur5k_annotations.json", "r") as f:
    imgur_data = json.load(f)

index_ids = imgur_data["index_id"]
index_to_ann_map = imgur_data["index_to_ann_map"]
ann_id = imgur_data["ann_id"]

imgur_samples = []

for idx in list(index_ids)[:1000]:
    ann_keys = index_to_ann_map[str(idx)]
    for ann_key in ann_keys:
        annotation = ann_id[ann_key]


        print("Annotation keys:", annotation.keys())


        break
    break


imgur_samples = []

for idx in list(index_ids)[:1000]:
    ann_keys = index_to_ann_map[str(idx)]
    for ann_key in ann_keys:
        annotation = ann_id[ann_key]


        image_key = "url" if "url" in annotation else "filename" if "filename" in annotation else None
        text_key = "text" if "text" in annotation else None

        if image_key is None or text_key is None:
            print("Unexpected annotation format:", annotation)
            continue

        imgur_samples.append({
            "image_url": annotation[image_key],
            "text": annotation[text_key]
        })

print(f"✅ Loaded {len(imgur_samples)} samples")


Annotation keys: dict_keys(['word', 'bounding_box'])
Unexpected annotation format: {'word': 'EXACTLY', 'bounding_box': '[605.33, 1150.67, 614.33, 226.33, 81.0]'}
Unexpected annotation format: {'word': 'WHERE', 'bounding_box': '[783.67, 654.67, 521.0, 222.33, 56.67]'}
Unexpected annotation format: {'word': 'I', 'bounding_box': '[959.0, 437.0, 76.67, 201.0, 38.33]'}
Unexpected annotation format: {'word': 'THROUGH', 'bounding_box': '[1966.0, 2446.33, 742.33, 289.0, -166.67]'}
Unexpected annotation format: {'word': 'THE', 'bounding_box': '[1503.0, 2453.0, 305.0, 240.0, 168.33]'}
Unexpected annotation format: {'word': 'DUST&', 'bounding_box': '[1179.33, 2311.33, 507.67, 261.0, 150.67]'}
Unexpected annotation format: {'word': 'WIND', 'bounding_box': '[870.67, 2052.67, 435.33, 222.33, 127.67]'}
Unexpected annotation format: {'word': 'UP', 'bounding_box': '[708.0, 1794.0, 221.67, 206.0, 116.0]'}
Unexpected annotation format: {'word': 'JUST', 'bounding_box': '[648.67, 1575.33, 332.67, 269.67, 1

In [9]:
def preprocess(example):
    if isinstance(example["image"], str):
        response = requests.get(example["image"], stream=True)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = example["image"].convert("RGB")

    encoding = processor(
        images=image,
        text=example["text"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    return {
        "pixel_values": encoding["pixel_values"].squeeze(0),
        "labels": encoding["labels"].squeeze(0)
    }


In [10]:
!pip install datasets transformers accelerate jiwer torchvision evaluate --quiet
import torch
from torch.utils.data import DataLoader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, get_scheduler
from datasets import load_dataset, Dataset
import evaluate
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import os
import evaluate

from torch.optim import AdamW
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.to(device)

from datasets import load_dataset


dataset = load_dataset("Teklia/IAM-line")


dataset = dataset["train"].train_test_split(test_size=0.1)
train_ds = dataset["train"]
test_ds = dataset["test"]


train_ds = train_ds.map(preprocess)
test_ds = test_ds.map(preprocess)


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Map:   0%|          | 0/5833 [00:00<?, ? examples/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

In [11]:
def collate_fn(batch):

    pixel_values = [example["pixel_values"] for example in batch]
    pixel_values = [torch.tensor(pv) if isinstance(pv, list) else pv for pv in pixel_values]
    pixel_values = torch.stack(pixel_values)


    labels = [example["labels"] for example in batch]
    labels = [torch.tensor(l) if isinstance(l, list) else l for l in labels]
    labels = torch.stack(labels)

    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

In [12]:
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)

In [13]:
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3
num_training_steps = epochs * len(train_loader)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [14]:
model.train()
for epoch in range(epochs):
    print(f"\n Epoch {epoch + 1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)


        model.config.pad_token_id = processor.tokenizer.pad_token_id
        model.config.decoder_start_token_id = processor.tokenizer.cls_token_id

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

model.save_pretrained("trocr-iam-imgur5k-finetuned")
processor.save_pretrained("trocr-iam-imgur5k-finetuned")
print("\n Model saved successfully.")


🚀 Epoch 1/3


Epoch 1: 100%|██████████| 1459/1459 [1:08:01<00:00,  2.80s/it, loss=0.137]



🚀 Epoch 2/3


Epoch 2: 100%|██████████| 1459/1459 [1:08:23<00:00,  2.81s/it, loss=0.122]



🚀 Epoch 3/3


Epoch 3: 100%|██████████| 1459/1459 [1:08:20<00:00,  2.81s/it, loss=0.109]



💾 Model saved successfully.


In [15]:
model.save_pretrained("trocr-iam-imgur5k-finetuned")
processor.save_pretrained("trocr-iam-imgur5k-finetuned")
print("\n Model saved successfully.")


💾 Model saved successfully.
